In [158]:
import numpy as np
from numpy.lib.stride_tricks import sliding_window_view
from itertools import product as quite_product
from tqdm.contrib.itertools import product
from collections import Counter
from itertools import cycle

from matplotlib import image
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as Data
import torchvision
import torchvision.transforms as transforms
from PIL import Image
import torchvision.transforms.functional as TF
from torchvision.utils import save_image
import sys
import matplotlib.pyplot as plt
import numpy as np
from math import ceil

In [159]:
weights_M  = 3
weights_N  = 3
ifmap_M = 224
ifmap_N = 224
ofmap_M = ifmap_M-weights_M+1
ofmap_N = ifmap_N-weights_M+1
ofmap_C = 1

In [114]:
pads = sliding_window_view(ifmap,window_shape=[1,weights_M,weights_N]).squeeze()
pads.shape

(3, 222, 222, 3, 3)

In [115]:
pad_origin = {}
for p_k, p_i, p_j in product(range(pads.shape[0]), range(pads.shape[1]), range(pads.shape[2])):
    for w_i, w_j in vanilla_product(range(pads.shape[3]), range(pads.shape[4])):
        cur_ifmap = pads[p_k, p_i, p_j, w_i, w_j]
        origin = {"pad_idx": (p_k, p_i, p_j), "weight_idx": (w_i, w_j)}
        if cur_ifmap not in pad_origin.keys():
            pad_origin[cur_ifmap] = [origin]
        else:
            pad_origin[cur_ifmap].append(origin)

100%|██████████| 147852/147852 [00:03<00:00, 40356.54it/s]


In [116]:
output = []
for f in quite_product(range(filters)):
    for k in quite_product(range(channels)):
        pad_completion_counter = Counter()
        for i, j in product(range(ifmap_M), range(ifmap_N)):
            output_added = False
            for assosciated_pads in pad_origin[ifmap[k][i][j]]:
                pad_idx = assosciated_pads['pad_idx']
                pad_completion_counter[pad_idx] += 1
                if pad_completion_counter[pad_idx] == 9:
                    output.append(np.einsum('ij,ij', pads[pad_idx], weights[f][k]))
                    output_added = True
            if i>0 and not output_added:
                output.append(0)
                    
        output.extend([0]*224)

output.extend([0])
output = np.array(output[1:]).reshape(filters,channels,ifmap_M,ifmap_N)

100%|██████████| 50176/50176 [00:00<00:00, 135938.37it/s]


In [119]:
calculated = np.sum(output, axis=1)

torch.Size([2, 224, 224])

In [132]:
[(torch.tensor(calculated[i]) == output[i]).all() for i in range(4)]

[tensor(True), tensor(True), tensor(True), tensor(True)]

In [186]:
def test(t = 1):
    def inner1():
        nonlocal t
        t += 1
        print(t)
    def inner2():
        nonlocal t
        print(t+1)
    inner1()
    inner2()
test()

2
3
